In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
from flask import Flask, send_file
import subprocess
import re

In [2]:
#For FAST NU LHR
URL = "http://lhr.nu.edu.pk/faculty/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
req = requests.get(URL, headers=headers)
print(req)

<Response [200]>


In [3]:
print(req.content)

b'<!DOCTYPE html>\n\n<html lang="en">\n\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta http-equiv="X-UA-Compatible" content="ie=edge">\n    \n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.2.1/css/bootstrap.min.css">\n    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/normalize/5.0.0/normalize.min.css">\n    <link rel=\'stylesheet\' href=\'https://unpkg.com/aos@2.3.0/dist/aos.css\'>\n    <link href="https://fonts.googleapis.com/css?family=Lato" rel="stylesheet">\n    <link href="https://fonts.googleapis.com/css?family=Lora" rel="stylesheet">\n\t<link href="https://fonts.googleapis.com/css?family=Roboto" rel="stylesheet">\n    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.7.1/css/all.css"\n        integrity="sha384-fnmOCqbTlWIlj8LyTjo7mOUStjsKC4pOpQbqyi7RrhN7udi9RwhKkMHpvLbHG9Sr" crossorigin="anonymous">\n    <link rel="stylesheet" type="t

In [4]:
soup = BeautifulSoup(req.content, 'html.parser') 
print(soup.prettify()) 

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="ie=edge" http-equiv="X-UA-Compatible"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.2.1/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="https://cdnjs.cloudflare.com/ajax/libs/normalize/5.0.0/normalize.min.css" rel="stylesheet"/>
  <link href="https://unpkg.com/aos@2.3.0/dist/aos.css" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lora" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Roboto" rel="stylesheet"/>
  <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.7.1/css/all.css" integrity="sha384-fnmOCqbTlWIlj8LyTjo7mOUStjsKC4pOpQbqyi7RrhN7udi9RwhKkMHpvLbHG9Sr" rel="stylesheet"/>
  <link href="/static/campus/CSS/stylesheet.css" rel="stylesheet" type="text/cs

In [5]:
f = soup.findAll('div', class_='facultyCard')
len(f)

184

In [6]:
id_=[]                                                           #int
name=[]                                                           #str
designation=[]                                                    #str
HECapproved=[]                                                  #bool
highesteducation=[]                                              #string
email=[]                                                        #email
department=[]                                                      #str
extension=[]                                                    #int
img=[]                                                           #URL


In [7]:
""""
This for loop is used to fetch necessary info about faculty staff
From FAST NU LHR

"""
for i in range(len(f)):
    faculty = f[i]
    try:
        id_.append(int(faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        id_.append(-1)
    name.append(faculty.find('h5', class_='text-center').text)
    designation.append(faculty.find('p',class_='small').text.strip().split('\n')[0])  #stripping to remove white spaces
    attribute_list = faculty.find('p', class_='small').text.strip().split('\n')
    filtered_list=[]
    for item in attribute_list:
            stripped_item = item.strip()  # Remove leading and trailing whitespace
            if stripped_item:  # if the stripped item is not an empty string/whitespace
                filtered_list.append(stripped_item) 
    if filtered_list.count("HEC Approved PhD Supervisor")==1:
            HECapproved.append(True);
    else:
            HECapproved.append(False)
    email.append(faculty.find('p', class_='mb-0').text)
    department.append(faculty.find('a',class_='faculty-link')['href'].split('/')[1])  #department is at index 1 not 0


    #tempURL to access each faculty profile
    tempURL='https://lhr.nu.edu.pk'+faculty.find('a',class_='faculty-link')['href']
    tempreq= requests.get(tempURL) 
    tempsoup = BeautifulSoup(tempreq.content, 'html.parser') 
    temp_faculty=tempsoup.find('span', class_='small')

    try:
        extension.append(int(temp_faculty.text.split(" ")[-1][4:]))
    except:
        extension.append(-1)

    temp_faculty = tempsoup.find('div', class_="text-justify")
    try:
        highest_edu_record=temp_faculty.find('li').text.split(",")[0]
        highest_edu_record = re.sub(r'[^a-zA-Z0-9(),.:;\s]', '', highest_edu_record)  #to remove special characters
        highesteducation.append(highest_edu_record)   
    except:
          highesteducation.append("No record")   
    img.append('https://lhr.nu.edu.pk'+faculty.find('img', class_='card-img-top')['src'])
    

In [8]:
df1 = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':designation,
                  'HEC Approved PHD Supervisor':HECapproved,
                  'Highest Education':highesteducation,
                  'Email':email,
                  'Department':department,
                  'Extension':extension,
                  'Image':img})


In [9]:
#This is done to match the format among different campuses 

df1.loc[df1['Department'] == 'fsc', 'Department'] = 'Department of Computer Science'
df1.loc[df1['Department'] == 'ee', 'Department'] = 'Department of Electrical Engineering'
df1.loc[df1['Department'] == 'cv', 'Department'] = 'Department of Civil Engineering'
df1.loc[df1['Department'] == 'fsm', 'Department'] = 'Fast School of Management'
df1.loc[df1['Department'] == 'ss', 'Department'] = 'Department of Sciences & Humanities'

In [10]:
df1.head()

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
0,1238,Dr. Kashif Zafar,Professor & HOD,True,Ph.D(Computer Science),kashif.zafar@nu.edu.pk,Department of Computer Science,221,https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._K...
1,4391,Dr. Asif Mahmood Gillani,Professor,True,Ph.D(Computing),asif.gilani@nu.edu.pk,Department of Computer Science,291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
2,5424,Dr. Hammad Naveed,Professor,True,Ph.D. (Bioengineering),hammad.naveed@nu.edu.pk,Department of Computer Science,227,https://lhr.nu.edu.pk/media/Faculty/01_-_Dr._H...
3,4027,Dr. Aamir Wali,Associate Professor,True,PhD (CS),aamir.wali@nu.edu.pk,Department of Computer Science,260,https://lhr.nu.edu.pk/media/Faculty/44_-_Dr._A...
4,6174,Dr. Arshad Ali,Associate Professor,True,PhD (CS),arshad.ali1@nu.edu.pk,Department of Computer Science,406,https://lhr.nu.edu.pk/media/Faculty/04_-_Dr._A...


In [11]:
df1.to_csv('lhr.csv' ,index=False)

In [12]:
folder_path = '.'  
subprocess.Popen(['explorer', folder_path], shell=True)

<Popen: returncode: None args: ['explorer', '.']>

In [13]:
id_=[]                                                    #int
name=[]                                                     #str
designation=[]                                            #str
HECapproved=[]                                               #bool
highesteducation=[]                                       #string
email=[]                                                     #email
department=[]                                                #str
extension=[]                                                 #int
img=[]                                                      #URL


In [14]:

URLS=["http://cfd.nu.edu.pk/department-cs/#faculty","https://cfd.nu.edu.pk/department-se/","https://cfd.nu.edu.pk/department-ee/#faculty","https://cfd.nu.edu.pk/department-fsm/#faculty","https://cfd.nu.edu.pk/department-sh/"]


In [15]:

""""
This for loop is used to fetch necessary info about faculty staff
From FAST NU CFD

"""

for j in range(len(URLS)):
        
        URL=URLS[j]
        #headers are used to have proper access to the websites
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        req = requests.get(URL, headers=headers)
        print(req)
        soup = BeautifulSoup(req.content, 'html.parser') 
        #print(soup.prettify()) 
        f = soup.findAll('div', class_="unitech-teacher")
        print(len(f))
        for i in range(len(f)):
                faculty=f[i]
                temp=(faculty.find('div',class_="unitech-teacher__inner"))
                name.append(temp.find('h4').text.strip())
                designation.append(temp.find('h6').text)
                img.append(faculty.find('div',class_="unitech-teacher__thumb").find('a').find('img')['src']) #src to get the link
                try:
                        value=faculty.find("ul",class_="unitech-teacher-social-icon").find("p",class_="hec").text
                        if("approved" in value):
                                HECapproved.append(True)
                except:
                        HECapproved.append(False)


                tempURL=faculty.find('div',class_="unitech-teacher__thumb")
                tempURL=tempURL.find('a')['href']
                tempreq= requests.get(tempURL) 
                tempreq = requests.get(tempURL, headers=headers)
                tempsoup = BeautifulSoup(tempreq.content, 'html.parser') 
                id_.append(int(tempsoup.find('link', attrs={'rel': 'shortlink'})["href"].split('=')[1]))

                try:
                        extension.append(int(tempsoup.find('ul', class_='teacher__address').find_all("li")[3].text.split(':')[1]))
                except:
                        extension.append(-1)
                department.append(tempsoup.find('ul', class_='teacher__address').find_all("li")[0].text.strip())
                email_id=(tempsoup.find('ul', class_='teacher__address').find_all("li")[1].text.strip())
                if('@' not in email_id):
                        email.append("No email id")
                else:
                        email.append(email_id)

                value
                
                try:
                        value=tempsoup.find("div",class_="htc__skill__container progress__bar--2").find("ul").find("li").text.replace('\u00A0',"").split("\n")[0] 
                except: 
                        value="No record"
                if(value=="No record" ):
                        try:
                                value=tempsoup.find("div",class_="htc__skill__container progress__bar--2").find("p").text.split("\n")[0]
                                if(value=='\u00A0'):
                                        value='No record'
                        except:
                                value="No record"
                if(value=="No record" ):
                        try:
                                value=tempsoup.find("div",class_="htc__skill__container progress__bar--2").find("p").text
                                if(value=='\u00A0'):
                                        value='No record'
                        except:
                                value="No record"
                if(value=="No record"):
                        try:
                                value=tempsoup.find("div",class_="htc__skill__container progress__bar--2").find_all("p")[1].text
                        except:
                                value="No record"
                if(value=="No record"):
                        try:
                                value=tempsoup.find("span",class_="m-list-timeline__time edu_course_title").text
                        except:
                                value="No record"
                if(value=="No record"):
                        try:
                                value=tempsoup.find("div",class_='htc__skill__container progress__bar--2').find("ol").find("li").text
                        except:
                                value="No record"
        

    
                value = re.sub(r'[^a-zA-Z0-9(),.:;\s]', '', value)  #does not allow sepcial characters
                value=re.sub(r'\u00C2', '', value)

                highesteducation.append(value)




        print("done with ",URLS[j])
        






        
        


<Response [200]>
49
done with  http://cfd.nu.edu.pk/department-cs/#faculty
<Response [200]>
15
done with  https://cfd.nu.edu.pk/department-se/
<Response [200]>
13
done with  https://cfd.nu.edu.pk/department-ee/#faculty
<Response [200]>
9
done with  https://cfd.nu.edu.pk/department-fsm/#faculty
<Response [200]>
20
done with  https://cfd.nu.edu.pk/department-sh/


In [16]:
df2 = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':designation,
                  'HEC Approved PHD Supervisor':HECapproved,
                  'Highest Education':highesteducation,
                  'Email':email,
                  'Department':department,
                  'Extension':extension,
                  'Image':img})


In [17]:
df2.head()

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
0,6074,Dr. Muhammad Ahmad,Associate Professor & HOD,True,Doctor of Philosophy (Hyperspectral Imaging) (...,dr.ahmad@nu.edu.pk,Department of Computer Science,160,https://cfd.nu.edu.pk/wp-content/uploads/2020/...
1,4748,Dr. Shahzad Sarfraz,Professor,True,"Ph.D. in Remote Sensing Image Processing AIT, ...",shahzad.sarfraz@nu.edu.pk,Department of Computer Science,102,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
2,507933,Dr. Anwar Shah,Assistant Professor,True,"PhD Computer Sciences, FASTNUCES, Peshawar",Anwar.Shah@nu.edu.pk,Department of Computer Science,131,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
3,4752,Dr. Bilal Khan,Assistant Professor,True,"Ph.D. (Computer Science Engineering), Dongguk...",khan.bilal@nu.edu.pk,Department of Computer Science,126,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
4,4772,Dr. Hashim Yasin,Assistant Professor,True,"Ph.D. (Computer Science), University of Bonn, ...",hashim.yasin@nu.edu.pk,Department of Computer Science,165,https://cfd.nu.edu.pk/wp-content/uploads/2019/...


In [18]:
df2.to_csv('cfd.csv', encoding='iso-8859-1', index=False)


In [19]:

#tto open folder using file explorer in the current directory 
folder_path = '.'  
subprocess.Popen(['explorer', folder_path], shell=True)

<Popen: returncode: None args: ['explorer', '.']>

In [20]:
id_=[]                                                     #int
name=[]                                                      #str
designation=[]                                                #str
HECapproved=[]                                      #bool
highesteducation=[]                                 #string
email=[]                                            #email
department=[]                                         #str
extension=[]                                        #int
img=[]                                              #URL


In [21]:
""""
This for loop is used to fetch necessary info about faculty staff
From FAST NU PWR

"""
for j in range(3):

    URL = ["http://pwr.nu.edu.pk/cs-faculty/","http://pwr.nu.edu.pk/ee-faculty/","http://pwr.nu.edu.pk/sh-faculty/"]
    req = requests.get(URL[j]) 
    #print(req) 
    soup = BeautifulSoup(req.content, 'html.parser') 
    #print(soup.prettify()) 
    f = soup.findAll('div', class_='team-members')
    temp=[]
    #print(len(f))
    import requests
    for i in range(len(f)):
        faculty=f[i]
        img.append('http://pwr.nu.edu.pk/'+faculty.find('img', class_='img-fullwidth')['src'])  #src to access iamge source
        id_.append(int(faculty.find('a',class_='text-theme-color-2')['href'].split('=')[1]))
        name.append((faculty.find('a',class_='text-theme-color-2').text.strip()))
        designation.append((faculty.find('h5',class_='text-theme-color').text))
        #tempURL to accesss each faculty profile
        tempURL='http://pwr.nu.edu.pk/'+faculty.find('a')['href']
        tempURL=tempURL.replace("https","http")
        tempreq= requests.get(tempURL) 
        tempsoup = BeautifulSoup(tempreq.content, 'html.parser') 
        department.append("Department of "+tempsoup.find_all('h5')[4].text)
        
        extension.append((int(tempsoup.find('li', class_='m-0').find('a',class_='text-gray').text.split('Ext')[1])))
        email.append(faculty.find('ul',class_='styled-icons').find('a')['href'].split(':')[1])

        div_elements=tempsoup.find_all('div',class_='container') #div_elements contains a list
        if(div_elements[4].find_all('h5')[1].text=='(HEC Approved)'): #5th elemnet is the desired one 
            HECapproved.append(True)
        else:
            HECapproved.append(False)
        value=(tempsoup.find("div",class_="tab-pane").find('ul',class_='list').find("li").text.split(",")[0])

        if(value=="#"):
            value="No record"
        value=value.replace("•	","")  #one record having double bullet points so we remove
        highesteducation.append(value)





In [22]:


df3 = pd.DataFrame({'ID':id_,
            'Name':name,
            'Designation':designation,
            'HEC Approved PHD Supervisor':HECapproved,
            'Highest Education':highesteducation,
            'Email':email,
            'Department':department,
            'Extension':extension,
            'Image':img})


In [23]:
df3.head()
        

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
0,1,Dr. Hafeez Ur Rehman,Associate Professor & HoD,True,Ph.D. in Computer and Control Engineering,hafeez.urrehman@nu.edu.pk,Department of Computer Science,107,http://pwr.nu.edu.pk/images/faculty/dr_hafeez.jpg
1,2,Dr. Omar Usman Khan,Associate Professor & Director,True,Ph.D. (Computer And Control Engineering),omar.khan@nu.edu.pk,Department of Computer Science,103,http://pwr.nu.edu.pk/images/faculty/Dr_Omer_Us...
2,3,Dr. Nouman Azam,Associate Professor,True,Ph.D. (CS),nouman.azam@nu.edu.pk,Department of Computer Science,134,http://pwr.nu.edu.pk/images/faculty/Dr_Noman_A...
3,4,Dr. Mohammad Nauman,Associate Professor,True,PhD(IT),mohammad.nauman@nu.edu.pk,Department of Computer Science,126,http://pwr.nu.edu.pk/images/faculty/Dr. Noman.jpg
4,5,Dr. Taimoor Khan,Assistant Professor,True,Ph.D. (Computer Science),taimoor.khan@nu.edu.pk,Department of Computer Science,127,http://pwr.nu.edu.pk/images/faculty/Dr_m_taimo...


In [24]:
df3.to_csv("pwr.csv",index=False)

In [25]:
folder_path = '.'  
subprocess.Popen(['explorer', folder_path], shell=True)

<Popen: returncode: None args: ['explorer', '.']>

In [26]:
#concatenating rows 
concat_df=pd.concat([df1,df2,df3],axis=0)

In [27]:
concat_df.head()

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
0,1238,Dr. Kashif Zafar,Professor & HOD,True,Ph.D(Computer Science),kashif.zafar@nu.edu.pk,Department of Computer Science,221,https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._K...
1,4391,Dr. Asif Mahmood Gillani,Professor,True,Ph.D(Computing),asif.gilani@nu.edu.pk,Department of Computer Science,291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
2,5424,Dr. Hammad Naveed,Professor,True,Ph.D. (Bioengineering),hammad.naveed@nu.edu.pk,Department of Computer Science,227,https://lhr.nu.edu.pk/media/Faculty/01_-_Dr._H...
3,4027,Dr. Aamir Wali,Associate Professor,True,PhD (CS),aamir.wali@nu.edu.pk,Department of Computer Science,260,https://lhr.nu.edu.pk/media/Faculty/44_-_Dr._A...
4,6174,Dr. Arshad Ali,Associate Professor,True,PhD (CS),arshad.ali1@nu.edu.pk,Department of Computer Science,406,https://lhr.nu.edu.pk/media/Faculty/04_-_Dr._A...


In [28]:
concat_df.to_csv("fast_faculty.csv",index=False)

In [29]:
folder_path = '.'  
subprocess.Popen(['explorer', folder_path], shell=True)

<Popen: returncode: None args: ['explorer', '.']>

In [30]:
faculty=pd.read_csv("fast_faculty.csv")

In [31]:
faculty

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
0,1238,Dr. Kashif Zafar,Professor & HOD,True,Ph.D(Computer Science),kashif.zafar@nu.edu.pk,Department of Computer Science,221,https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._K...
1,4391,Dr. Asif Mahmood Gillani,Professor,True,Ph.D(Computing),asif.gilani@nu.edu.pk,Department of Computer Science,291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
2,5424,Dr. Hammad Naveed,Professor,True,Ph.D. (Bioengineering),hammad.naveed@nu.edu.pk,Department of Computer Science,227,https://lhr.nu.edu.pk/media/Faculty/01_-_Dr._H...
3,4027,Dr. Aamir Wali,Associate Professor,True,PhD (CS),aamir.wali@nu.edu.pk,Department of Computer Science,260,https://lhr.nu.edu.pk/media/Faculty/44_-_Dr._A...
4,6174,Dr. Arshad Ali,Associate Professor,True,PhD (CS),arshad.ali1@nu.edu.pk,Department of Computer Science,406,https://lhr.nu.edu.pk/media/Faculty/04_-_Dr._A...
...,...,...,...,...,...,...,...,...,...
326,54,Mr. Osama Sohrab,Lecturer,False,M.S (Applied Mathematics),osama.sohrab@nu.edu.pk,Department of Science and Humanities,144,http://pwr.nu.edu.pk/images/faculty/Osama_Sohr...
327,53,Mr. Ikram Ullah,Assistant Professor,False,No record,ikram.ullah@nu.edu.pk,Department of Science and Humanities,145,http://pwr.nu.edu.pk/images/faculty/Ikram_Ulla...
328,52,Mr. Askar Ali,Lecturer,False,No record,askar.ali@nu.edu.pk,Department of Science and Humanities,145,http://pwr.nu.edu.pk/images/faculty/Askar_Ali1...
329,51,Ms. Noreen Shah,Lecturer,False,Ph.D. Scholar (Coursework completed),noreen.shah@nu.edu.pk,Department of Science and Humanities,150,http://pwr.nu.edu.pk/images/faculty/WhatsApp I...


In [32]:
sample_=faculty.sample(frac=0.8)   #student id last digit is 8 so 8/10=0.8

In [33]:
sample_

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image
309,62,Muhammad Usman,Instructor,False,Doing MS in Data Science at FAST- National Uni...,usman.rafiq@nu.edu.pk,Department of Computer Science,139,http://pwr.nu.edu.pk/images/faculty/Frame 88.jpg
145,6303,Ms. Sundas Munir,Lecturer,False,CA,sundas.munir@nu.edu.pk,Fast School of Management,-1,https://lhr.nu.edu.pk/media/Faculty/130-_Sunda...
203,506522,Mr. Ali Raza,Lecturer,False,Bachelors in Computer Science BS(CS) from FAS...,aliraza@nu.edu.pk,Department of Computer Science,122,https://cfd.nu.edu.pk/wp-content/uploads/2021/...
235,507218,Dr. Sajid Anwer,Assistant Professor,True,"PhD, Griffith University, Brisbane, Australia,...",sajid.anwer@nu.edu.pk,Department of Software Engineering,118,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
224,5549,Ms. Saba Ghani,Lecturer,False,"MS (Computer Science), FASTNU, CFD, Pakistan, ...",saba.ghani@nu.edu.pk,Department of Computer Science,169,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
...,...,...,...,...,...,...,...,...,...
1,4391,Dr. Asif Mahmood Gillani,Professor,True,Ph.D(Computing),asif.gilani@nu.edu.pk,Department of Computer Science,291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
291,2,Dr. Omar Usman Khan,Associate Professor & Director,True,Ph.D. (Computer And Control Engineering),omar.khan@nu.edu.pk,Department of Computer Science,103,http://pwr.nu.edu.pk/images/faculty/Dr_Omer_Us...
200,4842,Ms. Zain Iqbal,Assistant Professor,False,"M.S(Computer Science), FAST NUCES, Lahore, Pak...",zain.iqbal@nu.edu.pk,Department of Computer Science,-1,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
326,54,Mr. Osama Sohrab,Lecturer,False,M.S (Applied Mathematics),osama.sohrab@nu.edu.pk,Department of Science and Humanities,144,http://pwr.nu.edu.pk/images/faculty/Osama_Sohr...


In [34]:
sample_.to_csv("sample.csv",index=False)

In [35]:
folder_path = '.'  
subprocess.Popen(['explorer', folder_path], shell=True)

<Popen: returncode: None args: ['explorer', '.']>